In [ ]:
!pip install jb-caliban

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 3.0 MB/s eta 0:00:00


In [ ]:
pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 43.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp310-cp310-linux_x86_64.whl size=3542010 sha256=e900925ebef4b85d7df89eb617260c5f327c8ea8798f419d0374ff66e034880e
  Stored in directory: /root/.cache/pip/wheels/dc/52/e3/6c6b60b126b4d5c4370cb5ac071b82950f91649d62d72f7f56
Successfully built hdbscan


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
sys.path.insert(0, '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/analysis')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import hdbscan
import datetime

### Importing Created .py files

In [ ]:
import calculate_euclidean as ce
import interpolation as ip
import likelihood as li
import mark_outliers as mo
import mp4_details as mp4_d
import read_data_h5 as readH5
import visualize_coord as vc

In [ ]:
pd.set_option('display.max_columns', 500)
np.seterr(all = "ignore")
warnings.filterwarnings('ignore')

# 1. Pupil Diameter Analysis

In [ ]:
def master_pipeline(h5_file_path, subject_name, csv_path):
  '''
  The master pipeline takes the path of h5 file as a list and then calculates the diameter of the pupil for each subject using the euclidean distance.
  The function also calculates the area of the pupil and plots the visualization.

  Args:
    h5 file path (as a list)

  Return:
    master dataframe with all the information
  '''
  results = []

  for i, j, k in zip(h5_file_path, subject_name, csv_path):
    print('-'* 35, 'SUBJECT ID', j, '-'* 35)

    # Reading the .h5 file using the py file imported above
    df = readH5.read_df_from_h5(i, 'new')

    # Counting the total number of frames from the df
    overall_frame_count = len(df)

    # Defining the name of the coordinated column that will be used further by the user defined functions.
    coord_likelihood_pair = [ ['eye_top_x', 'eye_top_y', 'eye_top_likelihood'],
                           ['eye_bottom_x', 'eye_bottom_y', 'eye_bottom_likelihood'],
                           ['eye_right_x', 'eye_right_y', 'eye_right_likelihood'],
                           ['eye_left_x', 'eye_left_y', 'eye_left_likelihood'] ]
    
    # Interpolating the datapoints where the likelihood is less than the threshold
    new_df, total_interpolate = ip.interpolate_data(df, coord_likelihood_pair, 0.9)

    # Updating the resultant data into the original data
    df.update(new_df)
    display(df[['eye_top_likelihood', 'eye_bottom_likelihood', 'eye_left_likelihood', 'eye_right_likelihood']].describe())
    print()

    li.likelihood_plot(df)
    print()

    # Subsetting the above defined name of coordinates by elimating the likelihood column
    coord_pair = np.array(coord_likelihood_pair)[:, :2]

    # Calculating the diameter using the top and bottom coordinates
    df['eye_top_bottom_dist'] = df.apply(lambda x: ce.euclidean_distance(x['eye_top_x'], x['eye_top_y'], x['eye_bottom_x'], x['eye_bottom_y']), axis = 1)
    # Calculating the diameter using the right and left coordinates
    df['eye_right_left_dist'] = df.apply(lambda x: ce.euclidean_distance(x['eye_right_x'], x['eye_right_y'], x['eye_left_x'], x['eye_left_y']), axis = 1)

    # Approximating the diamter using the above two calcuated values
    df['eye_avg_diameter'] = ( df['eye_top_bottom_dist'] + df['eye_right_left_dist'] ) / 2.0
    # Calculating the area of the pupil
    df['eye_pupil_area'] = np.pi * ( df['eye_avg_diameter'] / 2) ** 2

    # Taking the average pupil diameter
    overall_final_pupil_diameter = df['eye_avg_diameter'].mean()
    # Taking the average pupil area
    overall_final_pupil_area = df['eye_pupil_area'].mean()

    df['frame_num'] = list(df.index)    
    
    # Marking the outliers from the dataframe and plotting them
    #df = mo.mark_outlier(df, coord_pair)
    df = mo.mark_outlier(df)

    # Fetching the mp4 details (#frames, duration, fps) from the mp4 file
    fps_i, num_frames_i, duration_i = mp4_d.get_mp4_details(i, 'new')

    print()
    # Calculate the time at which the frame was captured
    df['frame_time_sec'] = df.index  / fps_i
    # Calculating the minute part from the seconds
    df['frame_time_min'] = df['frame_time_sec'] // 60
    # Calculating the seconds part from the seconds
    df['frame_time_sec'] = df['frame_time_sec'] % 60

    # Rounding the mins and sec portion
    df['frame_time_sec'] = df['frame_time_sec'].round(2)
    df['frame_time_min'] = df['frame_time_min'].round(1)

    try:
      df.to_csv(k, index = False)
    except:
      pass

    display(df[df['clustering_outlier_detection'] == True])
    #outlier_df = df.copy()

    df = df[df['clustering_outlier_detection'] == False]

    # Visualizing the coordinates from the dataset
    vc.visualize_coords(df)
    print()

    # Taking the average pupil diameter
    overall_final_pupil_diameter = df['eye_avg_diameter'].mean()
    # Taking the average pupil area
    overall_final_pupil_area = df['eye_pupil_area'].mean()

    # Caluculating the outlier frames in the original data
    outlier_frames_count = overall_frame_count - len(df)
    
    ## Visualizing the change in diameter across the frames in the dataframe
    plt.figure(figsize = (12, 10))
    sns.lineplot(x = df.index, y = df['eye_avg_diameter'])
    skip = int(df['eye_avg_diameter'].max()) / int(df['eye_avg_diameter'].min()) 
    plt.yticks(range(int(df['eye_avg_diameter'].min()), int(df['eye_avg_diameter'].max()), int(skip)) )
    plt.show()

    ## Storing the relevant informations in a list
    row = [j, overall_final_pupil_diameter, overall_final_pupil_area, outlier_frames_count, total_interpolate, overall_frame_count,  (total_interpolate/overall_frame_count ) * 100, duration_i, fps_i, num_frames_i]
    results.append(row)
    print()
    print()

  # Creating the database using the nested results for each subject
  master_df = pd.DataFrame(results, columns = ['mouse_id', 'avg_pupil_diameter', 'avg_pupil_area', 'number_of_outlier_frames', 'num_of_interpolated_coord', 'overall_frames (from_df)', 'interpolated_proportion', 'mp4_duration (seconds)', 'frames_per_second', 'number_of_frames (from_cv2)'])
  
  return master_df

### sert40c_1

In [ ]:
all_tracking_file = ["/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40c_1_pre_2022-03-23-104712-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5",
                     "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40c_1_post1_2022-03-23-111933-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5",
                     "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40c_1_post2_2022-03-23-114729-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5"]

In [ ]:
all_tracking_subject = ["sert40c_1_pre_2022-03-23-104712-0000",
                        "sert40c_1_post1_2022-03-23-111933-0000",
                        "sert40c_1_post2_2022-03-23-114729-0000"]

In [ ]:
csv_path = ["/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323104712_840_baseline_video.csv",
            "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323111933_850_pcb1_video.csv",
            "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323114729_847_pcb2_video.csv"]

In [ ]:
final_df = master_pipeline(all_tracking_file, all_tracking_subject, csv_path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
final_df

,mouse_id,avg_pupil_diameter,avg_pupil_area,number_of_outlier_frames,num_of_interpolated_coord,overall_frames (from_df),interpolated_proportion,mp4_duration (seconds),frames_per_second,number_of_frames (from_cv2)
0,sert40c_1_pre_2022-03-23-104712-0000,61.629760,3024.828596,2,1708.0,5945,28.730025,588.025895,10.110099,5945
1,sert40c_1_post1_2022-03-23-111933-0000,55.126358,2425.137157,6,6104.0,12091,50.483831,1195.932901,10.110099,12091
2,sert40c_1_post2_2022-03-23-114729-0000,50.895905,2084.014839,14,4522.0,12093,37.393533,1196.130723,10.110099,12093


### sert40c_5

In [ ]:
all_tracking_file = ["/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40c_5_pre_2022-03-23-122109-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5",
                     "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40c_5_post1_2022-03-23-124848-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5",
                     "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40c_5_post2_ca3_2022-03-23-131423-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5",
                     "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40c_5_post3_2022-03-23-133041-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5"]

In [ ]:
all_tracking_subject = ["sert40c_5_pre_2022-03-23-122109-0000",
                        "sert40c_5_post1_2022-03-23-124848-0000",
                        "sert40c_5_post2_ca3_2022-03-23-131423-0000",
                        "sert40c_5_post3_2022-03-23-133041-0000"]

In [ ]:
csv_path = ["/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/ssert40c_5_pre_2022-03-23-122109_835_baseline_video.csv",
            "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_post1_2022-03-23-124848_830_pcb1_video.csv",
            "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_post2_ca3_2022-03-23-131423_828_pcb2_video.csv", 
            "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_post3_2022-03-23-133041_825_pcb3_video.csv"]

In [ ]:
final_df = master_pipeline(all_tracking_file, all_tracking_subject, csv_path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
final_df

,mouse_id,avg_pupil_diameter,avg_pupil_area,number_of_outlier_frames,num_of_interpolated_coord,overall_frames (from_df),interpolated_proportion,mp4_duration (seconds),frames_per_second,number_of_frames (from_cv2)
0,sert40c_5_pre_2022-03-23-122109-0000,53.140596,2250.974934,6,4589.0,5998,76.508836,593.268178,10.110099,5998
1,sert40c_5_post1_2022-03-23-124848-0000,54.517052,2352.974229,5,10303.0,12060,85.431177,1192.866660,10.110099,12060
2,sert40c_5_post2_ca3_2022-03-23-131423-0000,54.283102,2344.895087,1,5453.0,5983,91.141568,591.784513,10.110099,5983
3,sert40c_5_post3_2022-03-23-133041-0000,54.450348,2363.837976,12,13117.0,12033,109.008560,1190.196063,10.110099,12033


### sert40b_2

In [ ]:
all_tracking_file = ["/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40b_2_pre_2022-03-23-150530-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5",
                     "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40b_2_post1_2022-03-23-153252-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5",
                     "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40b_2_post2_ca3_2022-03-23-155746-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5",
                     "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert40b_2_post3_2022-03-23-161259-0000DLC_resnet50_PupillometrySep21shuffle1_947500.h5",]

In [ ]:
all_tracking_subject = ["sert40b_2_pre_2022-03-23-150530-0000",
                        "sert40b_2_post1_2022-03-23-153252-0000",
                        "sert40b_2_post2_ca3_2022-03-23-155746-0000",
                        "sert40b_2_post3_2022-03-23-161259-0000"]

In [ ]:
csv_path = ["/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323150533_811_baseline_video.csv",
            "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323153252_814_pcb1_video.csv",
            "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323155746_821_pcb2_video.csv",
            "/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323161259_818_pcb3_video.csv"]

In [ ]:
final_df = master_pipeline(all_tracking_file, all_tracking_subject, csv_path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
final_df

,mouse_id,avg_pupil_diameter,avg_pupil_area,number_of_outlier_frames,num_of_interpolated_coord,overall_frames (from_df),interpolated_proportion,mp4_duration (seconds),frames_per_second,number_of_frames (from_cv2)
0,sert40b_2_pre_2022-03-23-150530-0000,58.024974,2690.373237,1,2934.0,6020,48.737542,595.444220,10.110099,6020
1,sert40b_2_post1_2022-03-23-153252-0000,54.214327,2330.797113,7,14765.0,12058,122.449826,1192.668838,10.110099,12058
2,sert40b_2_post2_ca3_2022-03-23-155746-0000,55.361685,2443.791476,2,4947.0,6020,82.176080,595.444220,10.110099,6020
3,sert40b_2_post3_2022-03-23-161259-0000,57.695004,2644.325762,0,6926.0,6030,114.859038,596.433330,10.110099,6030


# 2. Behavioural Analysis

In [ ]:
import glob

In [ ]:
subjects = glob.glob('/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/*')

In [ ]:
subject_dict = {}

In [ ]:
for i in subjects:
  subject_name = i.split('/')[-1]
  subject_files = glob.glob(i + '/**.csv')
  subject_dict[subject_name] = subject_files

In [ ]:
subject_dict

{'sert40c_1': ['/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323104723_840_baseline.csv',
  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323111935_850_pcb1.csv',
  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323114731_847_pcb2.csv',
  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323104712_840_baseline_video.csv',
  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323111933_850_pcb1_video.csv',
  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323114729_847_pcb2_video.csv'],
 'sert40b_2': ['/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323150

In [ ]:
def analyze_behavior(data):
  data['category_velocity'] = np.nan
  data['lap_bin'] = data['lap_bin'].astype(str)
  
  data.loc[data['velocity'] >= 0.1, 'category_velocity'] = 'Movement'
  data.loc[data['velocity'] < 0.1, 'category_velocity'] = 'Stationary'

  fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (22, 8))
  sns.scatterplot(x = data.index, y = data['treadmillPosition'], hue = data['category_velocity'], ax = ax1)
  sns.lineplot(x = data.index, y = data['treadmillPosition'], hue = data['category_velocity'], ax = ax2)
  sns.scatterplot(x = data.index, y = data['treadmillPosition'], hue = data['lap_bin'], ax = ax3)

  plt.show()

In [ ]:
def test(df_l, df_r):

  df_l['frame_num'] = df_l.index

  df_r = df_r[['eye_avg_diameter', 'eye_pupil_area', 'frame_num', 'clustering_outlier_detection', 'frame_time_sec', 'frame_time_min']]
  roi_col = ['eye_avg_diameter', 'eye_pupil_area', 'velocity', 'treadmillPosition']

  final_df = pd.merge(df_l, df_r, on = 'frame_num', how = 'left')

  final_df['category_velocity'] = np.nan
  final_df['lap_bin'] = final_df['lap_bin'].astype(str)
  
  final_df.loc[final_df['velocity'] >= 0.1, 'category_velocity'] = 'Movement'
  final_df.loc[final_df['velocity'] < 0.1, 'category_velocity'] = 'Stationary'

  final_df['eye_pupil_area_normalized'] = final_df['eye_pupil_area'] / final_df['eye_pupil_area'].max()
  
  lap_endpoints = final_df.groupby(['lap_bin'], as_index = False).agg({'frame_num':'max'})['frame_num'].tolist()

  fig, axes = plt.subplots(2, 2, figsize = (22, 15))
  sns.scatterplot(data = final_df[(final_df['clustering_outlier_detection'] == False)], x = 'frame_num', y = 'eye_pupil_area', hue = 'category_velocity', ax = axes[0][0])
  sns.scatterplot(data = final_df[final_df['clustering_outlier_detection'] == False], x = 'frame_num', y = 'eye_pupil_area', hue = 'lap_bin', ax = axes[0][1])
  sns.lineplot(data = final_df[final_df['clustering_outlier_detection'] == False], x = 'frame_num', y = 'eye_pupil_area_normalized', ax = axes[1][0], color = 'red')
  sns.lineplot(data = final_df[final_df['clustering_outlier_detection'] == False], x = 'frame_num', y = 'treadmillPosition', ax = axes[1][0], color = 'black')
  sns.heatmap(final_df[roi_col].corr(), annot = True, ax = axes[1][1])

  for x_coord in lap_endpoints:
    axes[0][0].axvline(x = x_coord, linestyle = ':', color = 'black')

  plt.show()
  '''sns.scatterplot(data = final_df, x = 'frame_num', y = 'treadmillPosition', hue = 'category_velocity')
  plt.show()

  fig, axes = plt.subplots(4, 2, figsize = (22, 15))
  sns.scatterplot(data = final_df[final_df['lap_bin'] == '0'], x = 'frame_num', y = 'treadmillPosition', hue = 'category_velocity', ax = axes[0][0])
  sns.scatterplot(data = final_df[final_df['lap_bin'] == '1'], x = 'frame_num', y = 'treadmillPosition', hue = 'category_velocity', ax = axes[0][1])
  sns.scatterplot(data = final_df[final_df['lap_bin'] == '2'], x = 'frame_num', y = 'treadmillPosition', hue = 'category_velocity', ax = axes[1][0])
  sns.scatterplot(data = final_df[final_df['lap_bin'] == '3'], x = 'frame_num', y = 'treadmillPosition', hue = 'category_velocity', ax = axes[1][1])
  sns.scatterplot(data = final_df[final_df['lap_bin'] == '4'], x = 'frame_num', y = 'treadmillPosition', hue = 'category_velocity', ax = axes[2][0])
  sns.scatterplot(data = final_df[final_df['lap_bin'] == '5'], x = 'frame_num', y = 'treadmillPosition', hue = 'category_velocity', ax = axes[2][1])
  sns.scatterplot(data = final_df[final_df['lap_bin'] == '6'], x = 'frame_num', y = 'treadmillPosition', hue = 'category_velocity', ax = axes[3][0])
  sns.scatterplot(data = final_df[final_df['lap_bin'] == '7'], x = 'frame_num', y = 'treadmillPosition', hue = 'category_velocity', ax = axes[3][1])'''

In [ ]:
behavior = ['/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323150533_811_baseline.csv',
            '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323153257_814_pcb1.csv',
            '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323155750_821_pcb2.csv',
            '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323161302_818_pcb3.csv']

video = ['/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323150533_811_baseline_video.csv',
         '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323153252_814_pcb1_video.csv',
         '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323155746_821_pcb2_video.csv',
         '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40b_2/sert40b_2_20220323161259_818_pcb3_video.csv']

In [ ]:
for i, j in zip(behavior, video):
  subject_name = '_'.join(i.split('_')[-2:])
  temp_l = pd.read_csv(i)
  temp_r = pd.read_csv(j)

  print('-' * 75, subject_name, '-' * 75)
  test(temp_l, temp_r)
  print()
  print()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
behavior = ['/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323104723_840_baseline.csv',
            '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323111935_850_pcb1.csv',
            '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323114731_847_pcb2.csv',]

video = ['/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323104712_840_baseline_video.csv',
         '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323111933_850_pcb1_video.csv',
         '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_1/sert40c_1_20220323114729_847_pcb2_video.csv']

In [ ]:
for i, j in zip(behavior, video):
  subject_name = '_'.join(i.split('_')[-2:])
  temp_l = pd.read_csv(i)
  temp_r = pd.read_csv(j)

  print('-' * 75, subject_name, '-' * 75)
  test(temp_l, temp_r)
  print()
  print()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
behavior = ['/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_20220323122115_835_baseline.csv',
            '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_20220323124853_830_pcb1.csv',
            '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_20220323131430_828_pcb2.csv',
            '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_20220323133049_825_pcb3.csv']

video = ['/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/ssert40c_5_pre_2022-03-23-122109_835_baseline_video.csv',
         '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_post1_2022-03-23-124848_830_pcb1_video.csv',
         '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_post2_ca3_2022-03-23-131423_828_pcb2_video.csv',
         '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/behavior_data/sert40c_5/sert40c_5_post3_2022-03-23-133041_825_pcb3_video.csv']

In [ ]:
for i, j in zip(behavior, video):
  subject_name = '_'.join(i.split('_')[-2:])
  temp_l = pd.read_csv(i)
  temp_r = pd.read_csv(j)

  print('-' * 75, subject_name, '-' * 75)
  test(temp_l, temp_r)
  print()
  print()

Output hidden; open in https://colab.research.google.com to view.